# Part 1: Keras vs SKLearn's Linear Regression on Boston Housing

In [1]:

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4999195757502991947
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16378456385879714071
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8795739106509774324
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15884376474
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10469329121025213458
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
from sklearn.datasets import load_boston

boston = load_boston()

df = pd.DataFrame(boston.data)
df.columns = boston.feature_names
df['MEDV'] = boston.target
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## Data Scaling and Preprocessing

In [5]:
X = df
Y = boston.target

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X)
X_scaled = scaler.transform(X)

In [7]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [8]:
pd.DataFrame(X_scaled, columns=X.columns).head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562,0.159686
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439,-0.101524
2,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727,1.324247
3,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517,1.182758
4,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501,1.487503


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size = .25)

print("X:", X_train.shape, X_test.shape)
print("Y:", Y_train.shape, Y_test.shape)

X: (379, 14) (127, 14)
Y: (379,) (127,)


In [10]:
model = Sequential()
model.add(Dense(14, input_dim = 14, kernel_initializer = 'normal', activation = 'relu'))
model.add(Dense(1, kernel_initializer = 'normal'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mse'])
model


Instructions for updating:
Colocations handled automatically by placer.


In [11]:
model.fit(X_train, Y_train, batch_size = 1, epochs=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
379/379 [==============================] - 2s 5ms/step - loss: 526.1536 - mean_squared_error: 526.1536
Epoch 2/50
379/379 [==============================] - 1s 3ms/step - loss: 208.7040 - mean_squared_error: 208.7040
Epoch 3/50
379/379 [==============================] - 1s 3ms/step - loss: 57.1874 - mean_squared_error: 57.1874
Epoch 4/50
379/379 [==============================] - 1s 3ms/step - loss: 32.6305 - mean_squared_error: 32.6305
Epoch 5/50
379/379 [==============================] - 1s 3ms/step - loss: 25.1070 - mean_squared_error: 25.1070
Epoch 6/50
379/379 [==============================] - 1s 3ms/step - loss: 21.4137 - mean_squared_error: 21.4137
Epoch 7/50
379/379 [==============================] - 1s 3ms/step - loss: 19.0217 - mean_squared_error: 19.0217
Epoch 8/50
379/379 [==============================] - 1s 3ms/step - loss: 16.8142 - mean_squared_error: 16.8142
Epoch 9/50
379/379 [==============================] 

In [12]:
from sklearn.metrics import r2_score

Y_pred = model.predict(X_test)
r2_score(Y_test, Y_pred)

0.9962207465415054

In [13]:
# Predicted vs actual values
print([(pred, true) for (pred, true) in zip(Y_test.ravel()[:20], model.predict(X_test).ravel()[:20])])

[(27.1, 26.94757), (28.1, 28.003792), (19.4, 18.97062), (20.6, 20.203264), (33.8, 33.63118), (26.6, 27.129543), (8.5, 8.286227), (5.0, 6.788433), (28.0, 27.621101), (17.8, 17.486046), (20.3, 20.041935), (24.7, 24.990446), (17.0, 17.419622), (12.7, 13.136832), (20.0, 19.675642), (24.8, 24.814295), (18.5, 18.590433), (15.0, 15.126587), (21.4, 20.76298), (20.7, 20.249395)]


Comparing to SKLearn linear regression

https://github.com/RaymondDashWu/DS-2.1-Machine-Learning/blob/master/Boston_Housing_Prices_Linear_Regression.ipynb

1. <img src="https://i.imgur.com/KlQqTEH.png"></img>

Comparing to when I forgot to scale

In [14]:
X = df
Y = boston.target

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .25)

print("X:", X_train.shape, X_test.shape)
print("Y:", Y_train.shape, Y_test.shape)

X: (379, 14) (127, 14)
Y: (379,) (127,)


In [15]:
model = Sequential()
model.add(Dense(14, input_dim = 14, kernel_initializer = 'normal', activation = 'relu'))
model.add(Dense(1, kernel_initializer = 'normal'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mse'])
model.fit(X_train, Y_train, batch_size = 1, epochs=50)

Epoch 1/50
379/379 [==============================] - 1s 4ms/step - loss: 128.9923 - mean_squared_error: 128.9923
Epoch 2/50
379/379 [==============================] - 1s 3ms/step - loss: 56.2012 - mean_squared_error: 56.2012
Epoch 3/50
379/379 [==============================] - 1s 3ms/step - loss: 44.0247 - mean_squared_error: 44.0247
Epoch 4/50
379/379 [==============================] - 1s 3ms/step - loss: 36.0791 - mean_squared_error: 36.0791
Epoch 5/50
379/379 [==============================] - 1s 3ms/step - loss: 28.0295 - mean_squared_error: 28.0295
Epoch 6/50
379/379 [==============================] - 1s 3ms/step - loss: 20.2073 - mean_squared_error: 20.2073
Epoch 7/50
379/379 [==============================] - 1s 3ms/step - loss: 14.1995 - mean_squared_error: 14.1995
Epoch 8/50
379/379 [==============================] - 1s 3ms/step - loss: 9.7331 - mean_squared_error: 9.7331
Epoch 9/50
379/379 [==============================] - 1s 3ms/step - loss: 7.9629 - mean_squared_error: 7

In [16]:
Y_pred = model.predict(X_test)
r2_score(Y_test, Y_pred)

0.9999536068586035

In [17]:
print([(pred, true) for (pred, true) in zip(Y_test.ravel()[:20], model.predict(X_test).ravel().ravel()[:20])])

[(17.6, 17.533522), (19.4, 19.36438), (26.6, 26.621065), (33.1, 33.097595), (8.4, 8.300118), (23.2, 23.20561), (31.6, 31.530146), (22.0, 22.002716), (9.5, 9.436125), (23.3, 23.292746), (13.3, 13.227722), (37.0, 36.940887), (23.7, 23.683481), (37.9, 37.92071), (13.5, 13.478073), (20.8, 20.68146), (20.9, 20.805262), (23.1, 23.060043), (14.3, 14.245665), (37.2, 37.223278)]


# Part 2: Logistic Regression on Diabetes Dataset

In [18]:
from sklearn.datasets import load_diabetes

# diab = load_diabetes()
df_diab = pd.read_csv('../input/diabetes.csv')
# df_diab = pd.DataFrame(diab.data)
# df_diab.columns = diab.feature_names
df_diab.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [19]:
len(df_diab.columns)

9

## Scaling the data

In [20]:
X = np.array(df_diab)
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.46849198,
         1.4259954 ,  1.36589591],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.36506078,
        -0.19067191, -0.73212021],
       [ 1.23388019,  1.94372388, -0.26394125, ...,  0.60439732,
        -0.10558415,  1.36589591],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.68519336,
        -0.27575966, -0.73212021],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.37110101,
         1.17073215,  1.36589591],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.47378505,
        -0.87137393, -0.73212021]])

In [21]:
Y = df_diab['Outcome']

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size = .25)

print("X:", X_train.shape, X_test.shape)
print("Y:", Y_train.shape, Y_test.shape)

X: (576, 9) (192, 9)
Y: (576,) (192,)


In [22]:
model = Sequential()
model.add(Dense(9, input_dim = 9, kernel_initializer = 'normal', activation = 'relu'))
model.add(Dense(1, kernel_initializer = 'normal'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['accuracy'])
model.fit(X_train, Y_train, batch_size = 1, epochs=10)

Epoch 1/10
576/576 [==============================] - 2s 3ms/step - loss: 0.1126 - acc: 0.8594
Epoch 2/10
576/576 [==============================] - 2s 3ms/step - loss: 0.0021 - acc: 1.0000
Epoch 3/10
576/576 [==============================] - 2s 3ms/step - loss: 4.8819e-04 - acc: 1.0000
Epoch 4/10
576/576 [==============================] - 2s 3ms/step - loss: 2.5742e-04 - acc: 1.0000
Epoch 5/10
576/576 [==============================] - 2s 3ms/step - loss: 1.4849e-04 - acc: 1.0000
Epoch 6/10
576/576 [==============================] - 2s 3ms/step - loss: 9.6434e-05 - acc: 1.0000
Epoch 7/10
576/576 [==============================] - 2s 3ms/step - loss: 6.1782e-05 - acc: 1.0000
Epoch 8/10
576/576 [==============================] - 2s 3ms/step - loss: 4.3169e-05 - acc: 1.0000
Epoch 9/10
576/576 [==============================] - 2s 3ms/step - loss: 6.6380e-05 - acc: 1.0000
Epoch 10/10
576/576 [==============================] - 2s 3ms/step - loss: 1.6186e-04 - acc: 1.0000


In [23]:
Y_pred = model.predict(X_test)
r2_score(Y_test, Y_pred)

0.9994321479893653

In [24]:
print([(pred, true) for (pred, true) in zip(Y_test.ravel()[:20], model.predict(X_test).ravel().ravel()[:20])])

[(1, 1.0009819), (0, -0.004470244), (0, -0.003836602), (1, 0.9953842), (0, -0.00038474798), (0, -0.0011853501), (0, -0.0025659874), (1, 0.9992047), (0, -0.008324131), (0, -0.00076673925), (1, 0.9966607), (0, -0.00514476), (1, 1.0002908), (1, 0.99144125), (0, -0.0017786622), (1, 0.99348557), (0, -0.001224421), (0, -0.004188195), (1, 1.0022411), (0, -0.00082448125)]


Comparing to when I forgot to scale

In [25]:
model = Sequential()
model.add(Dense(9, input_dim = 9, kernel_initializer = 'normal', activation = 'relu'))
model.add(Dense(1, kernel_initializer = 'normal'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['accuracy'])
model.fit(X_train, Y_train, batch_size = 1, epochs=10)

Epoch 1/10
576/576 [==============================] - 2s 4ms/step - loss: 0.1175 - acc: 0.8507
Epoch 2/10
576/576 [==============================] - 2s 3ms/step - loss: 0.0031 - acc: 1.0000
Epoch 3/10
576/576 [==============================] - 2s 3ms/step - loss: 3.8168e-04 - acc: 1.0000
Epoch 4/10
576/576 [==============================] - 2s 3ms/step - loss: 1.7682e-04 - acc: 1.0000
Epoch 5/10
576/576 [==============================] - 2s 3ms/step - loss: 1.0940e-04 - acc: 1.0000
Epoch 6/10
576/576 [==============================] - 2s 3ms/step - loss: 6.7593e-05 - acc: 1.0000
Epoch 7/10
576/576 [==============================] - 2s 3ms/step - loss: 4.4776e-05 - acc: 1.0000
Epoch 8/10
576/576 [==============================] - 2s 3ms/step - loss: 2.2983e-05 - acc: 1.0000
Epoch 9/10
576/576 [==============================] - 2s 3ms/step - loss: 2.7716e-05 - acc: 1.0000
Epoch 10/10
576/576 [==============================] - 2s 3ms/step - loss: 5.6413e-05 - acc: 1.0000


In [26]:
print([(pred, true) for (pred, true) in zip(Y_test.ravel()[:20], model.predict(X_test).ravel()[:20])])

[(1, 1.0092651), (0, 0.0026898682), (0, 0.0055533797), (1, 1.0126312), (0, 0.0029559433), (0, 0.002331145), (0, -1.7568469e-05), (1, 1.0149949), (0, 0.0011534616), (0, 0.0028100014), (1, 1.0131356), (0, 0.0034810975), (1, 1.023494), (1, 1.0130394), (0, 0.0020176768), (1, 1.0196674), (0, -0.0016500726), (0, 0.0030111074), (1, 1.0071491), (0, 1.6540289e-06)]
